In [ ]:
import torch
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from smort.data.text_motion_dataset import TextMotionDataset
from smort.models.smort import SMORT
from smort.renderer.matplotlib import SingleMotionRenderer
from smort.rifke import feats_to_joints
from smort.data.collate import length_to_mask
from smort.models.text_encoder import TextToEmb

In [ ]:
from smort.data.data_module import InterXDataModule


text_motion_dataset = TextMotionDataset(
    "deps/interx/processed/dataset_2k.h5",
)
train_dataloader = DataLoader(
    text_motion_dataset,
    batch_size=1,
    collate_fn=text_motion_dataset.collate_fn,
    shuffle=True,
    # num_workers=7,
    # persistent_workers=True,
)

data_module = InterXDataModule(
    "deps/interx/processed/dataset_2k.h5",
    batch_size=1,
    num_workers=1,
    use_tiny=True,
    return_scene=True,
)

# next(iter(train_dataloader))

In [ ]:
mean, std = text_motion_dataset.get_mean_std()
assert type(mean) == torch.Tensor and type(std) == torch.Tensor
model = SMORT(mean, std)

trainer = Trainer(
    accelerator="cpu", max_epochs=10, fast_dev_run=False, num_sanity_val_steps=0
)

trainer.fit(model, data_module)

In [ ]:
# # model = SMORT.load_from_checkpoint(
# #     "lightning_logs/version_21/checkpoints/epoch=11-step=1200.ckpt",
# # )
# text_embeds = TextToEmb(
#     "distilbert/distilbert-base-uncased",
#     device=model.device,
# )(
#     [
#         "Two people walk towards each other. "
#         "After they meet, the first person hugs the second person around the "
#         "shoulders, gently patting his/her back with his/her right hand. "
#         "Meanwhile, the second person puts his/her arms around the first "
#         "person's waist and pats his/her waist with his/her right hand."
#     ]
# )
# # text_embeds

# mask = length_to_mask(text_embeds["length"], device=model.device)
# encoded = model.text_encoder(
#     {
#         "x": text_embeds["x"],
#         "mask": mask,
#     }
# )

# dists = encoded.unbind(1)
# mu, logvar = dists
# latent_vectors = mu
# motion = text_motion_dataset.reverse_norm(
#     model.motion_decoder(
#         {
#             "z": latent_vectors,
#             "mask": mask,
#         }
#     ).squeeze(dim=0)
# )

# renderer = SingleMotionRenderer(
#     colors=("red", "red", "red", "red", "red"),
# )

# renderer.render_animation_single(
#     feats_to_joints(torch.from_numpy(motion)).detach().cpu().numpy()
# )